## 3.1.5 Realistic simulation setup
Script for the case of having a spot centered inbetween 4 pixels and the spot is double the size

### Plot spot with pixel diameter
- ~~create a mesh with pixel size distance corners~~
- ~~create a spot centered in the middle of the plane~~
- ~~Pass the spot through the mesh~~ 
- ~~Each pixel that has the spot goes to a micro mesh~~
- ~~Each micropixel that has the spot sum.~~
- ~~Consider the case where the spot is inbetween two pixels~~
    - ~~Expand the mesh by 1 in each direction~~
- ~~Consider the case where the spot is inbetween 4 pixels~~
- Consider the case where:
    - ~~spot is double the size centered inbetween  4 pixels~~
    - spot is centered in the middle of a pixel

### import  libraries

In [1]:
vscode = 1

In [2]:
if(vscode == 0):
    # for vscode
    # for jupyter notebook
    from mpl_toolkits.mplot3d import axes3d
    import matplotlib.pyplot as plt

    %matplotlib notebook
elif(vscode == 1):
    %matplotlib qt
    

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 
from typing import Sequence
from calib_lib import *
from matplotlib.collections import PatchCollection      # pathcoletion for multiple patches
import matplotlib.cm as cm                              # for colormaps
from matplotlib.patches import Rectangle                # for rectangle representation



DECIMALS = 2            # how many decimal places to use in print


In [4]:
F = 16                                       # focal length( in mm )
image_size = np.array([1936,1216])               # sensor size(in mm)
#image_size = np.array([11.345,7.126])
pixel_width = 5.86e-3                       # pixel size in mm
PX= image_size[0]/2.0                       # principal point x-coordinate
PY= image_size[1]/2.0                       # principal point y-coordinate
IMAGE_HEIGTH = image_size[1]
IMAGE_WIDTH = image_size[0]
THETA_X = 0                                 # roll angle
THETA_Y = 0                                 # pitch angle
THETA_Z = 0                         # yaw angle


# camera Right
THETA_X_R = 0                                 # roll angle
THETA_Y_R = 0                                 # pitch angle
THETA_Z_R= 0                                 # yaw angle
# camera Left
THETA_X_L = 0                                 # roll angle
THETA_Y_L = 0                                 # pitch angle
THETA_Z_L= 0                                 # yaw angle

C_L = np.array([0,0,0])                     # camera centre
C_R = np.array([500,0,0])

chess_dimx,chess_dimy = (13,9)
chess_sq_size = 44

## Spot between 4 pixels without touching the corners
- consider the centroid and mark the pixels

### Spot 

In [5]:
# Generate pixel grid
pixelGridX  = (np.arange(0,image_size[0])) * pixel_width
pixelGridY  = (np.arange(0,image_size[1])) * pixel_width
pixelGridXX, pixelGridYY = np.meshgrid(pixelGridX,pixelGridY)

In [6]:
def spot_center_coord(cx0,cy0,pixelGridX,pixelGridY):
    cx0_max = np.where((cx0 < pixelGridX))
    cx0_min = np.where((cx0 > pixelGridX))
    #print(cx0_max[0][0])
    #print(cx0_min[0][-1])
    cy0_max = np.where((cy0 < pixelGridY))
    cy0_min = np.where((cy0 > pixelGridY))
    #print(cy0_max[0][0])
    #print(cy0_min[0][-1])

    return cx0_max[0][0],cx0_min[0][-1],cy0_max[0][0],cy0_min[0][-1]

In [7]:
#spot centroid coordinates 
cx0 = 5.675
cy0 = 3.5630

In [88]:
def pixel_cover_finder(gridSize,cx0max,cy0max,cx0,cy0,r):
    # array for percentage storage
    pixel_covered_percent = np.zeros((1,))

    xgrid = np.linspace(0,1,gridSize)*pixel_width
    ygrid = np.linspace(0,1,gridSize)*pixel_width

    x_pixel = cx0max+xgrid
    y_pixel = cy0max+ygrid

    xx_pixel, yy_pixel = np.meshgrid(x_pixel,y_pixel)

    circ_total = np.sqrt((xx_pixel-cx0)**2 + (yy_pixel-cy0)**2) <= r
    circ = np.where(circ_total == True)

    pixel_covered = circ[0].shape
    totalpixel_area = gridSize**2
    pixel_covered_percent = (pixel_covered[0]/totalpixel_area)*100
    print("pixel cover percentage: " + str(pixel_covered_percent) + " %")

    return pixel_covered_percent

In [89]:
# the objective is, knowing the radius, create an area to be inspected. The difficulty stands in having the whole object in the area.
cx0max,cx0min,cy0max,cy0min = spot_center_coord(cx0,cy0,pixelGridX,pixelGridY)
cx0min*pixel_width

5.67248

In [98]:
gridSize = 1024
theta = np.linspace(0, 2*np.pi, gridSize)
zoom = 10
r = pixel_width*zoom
x0 = r*np.cos(theta)+(cx0)
y0 = r*np.sin(theta)+(cy0)

fig, ax = plt.subplots()
ax.plot(pixelGridXX,pixelGridYY)
ax.plot(pixelGridXX.T,pixelGridYY.T)
ax.plot(x0,y0, linewidth=1)
#ax.plot(pixelGridX[circ_corners[1][:]],pixelGridY[circ_corners[0][:]],'r*')
ax.plot(cx0,cy0,'k.')
ax.set_xlim([cx0-(zoom*pixel_width),cx0+(zoom*pixel_width)])
ax.set_ylim([cy0-(zoom*pixel_width),cy0+(zoom*pixel_width)])
ax.set_aspect(1)
plt.show()

In [99]:
cx0max_pxl = []
cx0min_pxl = []
cy0max_pxl = []
cy0min_pxl = []
points_max_min = []
for i in range(x0.shape[0]):
    cx0max,cx0min,cy0max,cy0min = spot_center_coord(x0[i],y0[i],pixelGridX,pixelGridY)

    # max and min points in (x,y) coordinates
    cx0min_pxl.append(cx0min*pixel_width)
    cx0max_pxl.append(cx0max*pixel_width)
    cy0min_pxl.append(cy0min*pixel_width)
    cy0max_pxl.append(cy0max*pixel_width)

    #array max and min points
    points_max_min.append(([cx0min_pxl[i],cy0min_pxl[i]]))

In [100]:
# remove duplicates
#list for the final values
points_deduplt = list()

[points_deduplt.append(item) for item in points_max_min if item not in points_deduplt]
print(np.shape(points_deduplt))

(80, 2)


In [101]:
cx0max,cx0min,cy0max,cy0min = spot_center_coord(cx0,cy0,pixelGridX,pixelGridY)

# max and min points in (x,y) coordinates
cx0min_pxl = cx0min*pixel_width
cx0max_pxl = cx0max*pixel_width
cy0min_pxl = cy0min*pixel_width
cy0max_pxl = cy0max*pixel_width

#array max and min points
center_max_min = np.array([[cx0min_pxl,cy0min_pxl],
                            [cx0min_pxl,cy0max_pxl],
                            [cx0max_pxl,cy0min_pxl],
                            [cx0max_pxl,cy0max_pxl]])

In [102]:
points_max_min = np.array(points_max_min)

In [103]:
circ1 = np.sqrt((pixelGridXX-cx0)**2 + (pixelGridYY-cy0)**2) <= r
circ_corners = np.where(circ1 == True)

In [104]:
fig,ax = plt.subplots()
#pixel_orient = np.array([[1,1],[1,-1],[-1,1],[-1,-1]])
# center of the pixel
ax.plot(cx0,cy0,'k.')
#axis
ax.plot(pixelGridXX,pixelGridYY,'r')
ax.plot(pixelGridXX.T,pixelGridYY.T,'b')
#inner corners
#ax.plot(pixelGridX[circ_corners[1][:]],pixelGridY[circ_corners[0][:]],'r*')
#spot
ax.plot(x0,y0,'r',linewidth=4)
# list for the patches
ptchs =[]
#pixel cover list
pixel_covered_percent = []
for j in range(len(points_deduplt)):
    pixel_covered_percent.append(pixel_cover_finder(gridSize,points_deduplt[j][0],points_deduplt[j][1],cx0,cy0,r))
    ptchs.append(Rectangle((points_deduplt[j][0],points_deduplt[j][1]), pixel_width,pixel_width))

""" for i in range(len(circ_corners[0])):
    pixel_covered_percent.append(pixel_cover_finder(gridSize,circ_corners[1][i]*pixel_width,circ_corners[0][i],cx0,cy0,r))
    ptchs.append(Rectangle((circ_corners[1][i]*pixel_width,circ_corners[0][i]*pixel_width), pixel_width,pixel_width))
 """
p = PatchCollection(ptchs, cmap=cm.jet, alpha=1)
pixel_covered_percent_arr = np.array(pixel_covered_percent).flatten()
# define the color
p.set_array(pixel_covered_percent_arr)
# set color max and min
p.set_clim([0, 100])
ax.add_collection(p)

fig.colorbar(p)

ax.set_xlim([cx0-((zoom+1)*pixel_width),cx0+((zoom+1)*pixel_width)])
ax.set_ylim([cy0-((zoom+1)*pixel_width),cy0+((zoom+1)*pixel_width)])

ax.set_aspect(1)

plt.show()

pixel cover percentage: 41.43962860107422 %
pixel cover percentage: 31.584453582763672 %
pixel cover percentage: 11.444950103759766 %
pixel cover percentage: 99.90062713623047 %
pixel cover percentage: 79.97875213623047 %
pixel cover percentage: 36.489200592041016 %
pixel cover percentage: 0.9052276611328125 %
pixel cover percentage: 77.88228988647461 %
pixel cover percentage: 12.67547607421875 %
pixel cover percentage: 91.06311798095703 %
pixel cover percentage: 17.018890380859375 %
pixel cover percentage: 88.30299377441406 %
pixel cover percentage: 7.6152801513671875 %
pixel cover percentage: 63.33026885986328 %
pixel cover percentage: 99.27701950073242 %
pixel cover percentage: 15.668678283691406 %
pixel cover percentage: 53.26986312866211 %
pixel cover percentage: 79.91476058959961 %
pixel cover percentage: 95.82557678222656 %
pixel cover percentage: 0.018787384033203125 %
pixel cover percentage: 1.6480445861816406 %
pixel cover percentage: 0.1361846923828125 %
pixel cover percenta

In [86]:
points_deduplt

[[5.6783399999999995, 3.56288],
 [5.67248, 3.56288],
 [5.67248, 3.56874],
 [5.66662, 3.56288],
 [5.66662, 3.55702],
 [5.67248, 3.55702],
 [5.6783399999999995, 3.55702]]

### Number of pixels inbetween

In [ ]:
nr_pixel_x = ((cx0max*pixel_width)-(cx0min*pixel_width))/pixel_width
print("number of pixels in the x axis with the spot:",np.round(nr_pixel_x))
nr_pixel_y = ((cy0max*pixel_width)-(cy0min*pixel_width))/pixel_width
print("number of pixels in the y axis with the spot:", np.round(nr_pixel_y))

## Plot everything

In [ ]:
def pixel_cover_finder(gridSize,cx0max,cy0max,cx0,cy0,r):

    # array for percentage storage
    pixel_covered_percent = np.zeros((4,))

    xgrid = np.linspace(0,1,gridSize)*pixel_width
    ygrid = np.linspace(0,1,gridSize)*pixel_width

    x_pixel = (cx0max)+(xgrid)
    y_pixel = (cy0max)+(ygrid)

    xx_pixel, yy_pixel = np.meshgrid(x_pixel,y_pixel)

    circ_total = np.sqrt((xx_pixel-cx0)**2 + (yy_pixel-cy0)**2) <= r
    circ = np.where(circ_total == True)

    pixel_covered = circ[0].shape
    totalpixel_area = gridSize**2
    pixel_covered_percent = (pixel_covered[0]/totalpixel_area)*100
    #print("pixel cover percentage: " + str(pixel_covered_percent[i]) + " %")

    return pixel_covered_percent

    

In [ ]:
fig,ax = plt.subplots()
pixel_orient = np.array([[1,1],[1,-1],[-1,1],[-1,-1]])
# center of the pixel
ax.plot(cx0,cy0,'k.')
#axis
ax.plot(pixelGridXX,pixelGridYY,'r')
ax.plot(pixelGridXX.T,pixelGridYY.T,'b')
#spot
ax.plot(x0,y0, linewidth=1)
# list for the patches
ptchs =[]
#pixel cover list
pixel_covered_percent = []
for j in range(len(points_max_min)):
    pixel_covered_percent.append(pixel_cover_finder(gridSize,points_max_min[j,0],points_max_min[j,1],cx0,cy0,r))
    for i in range(4):
        ptchs.append(Rectangle((points_max_min[j,0],points_max_min[j,1]), pixel_orient[i,0]*pixel_width,pixel_orient[i,1]*pixel_width))


for j in range(len(circ_corners[1][:])):
    pixel_covered_percent.append(pixel_cover_finder(gridSize,pixelGridX[circ_corners[1][j]],pixelGridY[circ_corners[0][j]],cx0,cy0,r))
    for i in range(4):
        ptchs.append(Rectangle((pixelGridX[circ_corners[1][j]],pixelGridY[circ_corners[0][j]]), pixel_orient[i,0]*pixel_width,pixel_orient[i,1]*pixel_width))

p = PatchCollection(ptchs, cmap=cm.jet, alpha=1)
pixel_covered_percent_arr = np.array(pixel_covered_percent).flatten()
# define the color
p.set_array(pixel_covered_percent_arr)
# set color max and min
p.set_clim([0, 100])
ax.add_collection(p)

fig.colorbar(p)

ax.set_xlim([cx0-(zoom*pixel_width),cx0+(zoom*pixel_width)])
ax.set_ylim([cy0-(zoom*pixel_width),cy0+(zoom*pixel_width)])

ax.set_aspect(1)

plt.show()